In [1]:
import numpy as np
import scipy.linalg
import scipy.sparse as sp

In [2]:
def jcf(A):
    n = len(A)
    W = []
    S = []
    P = []
    J_rows = []
    J_cols = []
    J_vals = []
    diag_ptr = 0
    eigvals, cnts = np.unique(
        np.round(np.linalg.eigvals(A), decimals=6),
        return_counts=True)
    for eigval, cnt in zip(eigvals, cnts):
        mat = eigval * np.eye(n) - A
        S = scipy.linalg.null_space(mat)
        basis_nums = [S.shape[1]]
        mats = [mat]
        while basis_nums[-1] < cnt:
            mat = (eigval * np.eye(n) - A).dot(mat)
            mats.append(mat)
            S, _ = np.linalg.qr(np.hstack(
                (S, scipy.linalg.null_space(mat))))
            S = S[:, np.where(np.all(
                np.abs(mat.dot(S)) < 1e-6, axis=0))[0]]
            basis_nums.append(S.shape[1])
        basis_nums_arr = np.array(basis_nums)
        block_sz_atleast = np.insert(
            basis_nums_arr[1: ] - basis_nums_arr[: -1],
            0, basis_nums_arr[0])
        block_sz = np.insert(
            block_sz_atleast[: -1] - block_sz_atleast[1: ],
            len(block_sz_atleast) - 1, block_sz_atleast[-1])
        k = len(block_sz)
        basis_nums.insert(0, 0)
        for i, sz in enumerate(np.flip(block_sz)):
            if i == 0:
                assert sz == \
                    basis_nums[k - i] - basis_nums[k - 1 - i]
                T = S[:, basis_nums[k - 1 - i]: ]
                for idx in range(basis_nums[k - 1 - i],
                                 basis_nums[k - i]):
                    v = S[:, idx]
                    P_reverse = []
                    for j in range(k - i):
                        P_reverse.append(v)
                        v = mats[0].dot(v)
                        if j == 0:
                            J_rows.append(diag_ptr)
                            J_cols.append(diag_ptr)
                            J_vals.append(eigval)
                        else:
                            J_rows.extend([diag_ptr - 1, diag_ptr])
                            J_cols.extend([diag_ptr, diag_ptr])
                            J_vals.extend([-1, eigval])
                        diag_ptr += 1
                    P_reverse.reverse()
                    P.extend(P_reverse)
            else:
                num_pre_basis = T.shape[1]

                T, _ = np.linalg.qr(np.hstack((
                    mats[0].dot(T),
                    S[:, basis_nums[k - 1 - i]: basis_nums[k - i]]
                )))
                T = T[:, np.where(np.all(
                    np.abs(mats[k - i - 1].dot(T)) < 1e-6,
                    axis=0))[0]]

                assert sz == T.shape[1] - num_pre_basis
                for idx in range(num_pre_basis, T.shape[1]):
                    v = T[:, idx]
                    P_reverse = []
                    for j in range(k - i):
                        P_reverse.append(v)
                        v = mats[0].dot(v)
                        if j == 0:
                            J_rows.append(diag_ptr)
                            J_cols.append(diag_ptr)
                            J_vals.append(eigval)
                        else:
                            J_rows.extend([diag_ptr - 1, diag_ptr])
                            J_cols.extend([diag_ptr, diag_ptr])
                            J_vals.extend([-1, eigval])
                        diag_ptr += 1
                    P_reverse.reverse()
                    P.extend(P_reverse)
    J = sp.coo_matrix((J_vals, (J_rows, J_cols)),
                      shape=(diag_ptr, diag_ptr)).A
    P = np.vstack(P).T
    return J, P

In [3]:
A = np.array([
    [3., -1., 0., 0.],
    [1., 1., 0., 0.],
    [0., 0., 2., 0.],
    [-6., 6., -3., 3.]
])

In [4]:
J, P = jcf(A)

In [5]:
print(np.abs(A - P.dot(J.dot(np.linalg.inv(P)))).max()
      / np.abs(A).max())

5.83237162269749e-14
